<a href="https://colab.research.google.com/github/Rweissman7/Rweissman7.github.io/blob/main/mysql_Finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ruchel Weissman  
**SQL Queries**  
**Northwinds Database**

**Introduction**  
The Northwinds database is a widely used sample database designed by Microsoft for
educational and testing purposes. It simulates a company's operations and comprises
multiple tables, including products, customers, suppliers, employees, orders, and
categories. The data within these tables is fictional but representative of real-world
scenarios, making it an ideal resource for learning and practicing relational database
concepts. With its well-structured schema and relationships, the Northwinds database
serves as an excellent tool for understanding data organization, SQL queries, data
manipulation, and database management in various platforms such as Microsoft Access
and other database management systems. I used python with mysql connector to pull and
aggregate queries from the database.

In [ ]:

import mysql.connector

In [ ]:
#connect to database
try:
    db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="Esty5994453",
    database="northwind"
    )
except mysql.connector.Error as error:
    print("Error connecting to the database:", error)

In [ ]:
#set cursor use buffered not to get "Error executing the query: Unread result found"
mycursor = db.cursor(buffered=True)

In [ ]:
#create database testdb
try:
    mycursor.execute("CREATE DATABASE northwind")
except mysql.connector.Error as error:
    print("Error executing the query:", error)

Error executing the query: 1007 (HY000): Can't create database 'northwind'; database exists


**Data Profiling**  
Using SQL queries, I performed data profiling on the dataset, analyzing customer spending
patterns in the "Orders" and "OrderDetails" tables. By calculating the average order value
for each customer, I gained insights into their behavior. This analysis enabled me to identify
high-value customers who consistently placed large orders. These findings can inform
tailored marketing strategies and drive business growth.

In [ ]:
#1 Customer spending patterns: Average order value for each customer
import pandas as pd

try:
    mycursor.execute('''SELECT ShipName, ROUND(AVG((UnitPrice*Quantity)-Discount),2) AS AvgOrderValue
    FROM orders
    NATURAL JOIN orderdetails
    GROUP BY ShipName
    ORDER BY AvgOrderValue''')

    title = [i[0] for i in mycursor.description]
    #print(title)
    data=[]
    for x in mycursor:
        data.append(x)
    df = pd.DataFrame(data, columns=[title])
    print(df)
except mysql.connector.Error as error:
    print("Error executing the query:", error)

                         ShipName AvgOrderValue
0      Centro comercial Moctezuma         50.40
1   Laughing Bacchus Wine Cellars         65.31
2            Galera del gastronmo        104.59
3                Romero y tomillo        104.81
4                     North/South        108.17
..                            ...           ...
85                  Hanari Carnes       1065.66
86                   Ernst Handel       1110.16
87               Piccolo und mehr       1141.74
88                  Simons bistro       1209.23
89                     QUICK-Stop       1366.09

[90 rows x 2 columns]


**Creating Views**  
Employing SQL commands, I crafted a view to offer distinct perspectives on the data. This
view showcases the total revenue generated by each customer. By joining the "Customers,"
"Orders," and "OrderDetails" tables, I calculated the total revenue generated by individual
customers. The view includes informative columns such as customer name, customer ID,
and their corresponding total revenue. This view not only provides valuable insights into
each customer's contribution to overall revenue, but it also streamlines the process of
pulling up this exact query at any time, making data analysis and reporting more efficient
and accessible.

In [ ]:
#2 view that shows total revenue generated by each customer
try:
    mycursor.execute('''CREATE VIEW TotalRevenue AS
    SELECT customers.CompanyName, customers.CustomerID, ROUND(SUM((UnitPrice*Quantity)-Discount),2) AS TotalRevenue
    FROM orders
    JOIN orderdetails ON orders.OrderID = orderdetails.OrderID
    JOIN customers ON orders.CustomerId = customers.CustomerID
    GROUP BY customers.CompanyName, customers.CustomerID''')

except mysql.connector.Error as error:
    print("Error executing the query:", error)

Error executing the query: 1050 (42S01): Table 'TotalRevenue' already exists


In [ ]:
#view total revenue view
try:
    mycursor.execute("SELECT*FROM TotalRevenue")

    title = [i[0] for i in mycursor.description]
    data=[]
    for x in mycursor:
        data.append(x)
    df = pd.DataFrame(data, columns=[title])
    print(df)

except mysql.connector.Error as error:
    print("Error executing the query:", error)

                           CompanyName CustomerID TotalRevenue
0                  Alfreds Futterkiste      ALFKI      4596.20
1   Ana Trujillo Emparedados y helados      ANATR      1402.95
2               Antonio Moreno Taquera      ANTON      7515.35
3                      Around the Horn      AROUT     13806.50
4                        B's Beverages      BSBEV      6089.90
..                                 ...        ...          ...
84                      Wartian Herkku      WARTH     16617.10
85              Wellington Importadora      WELLI      6480.70
86                White Clover Markets      WHITC     29073.45
87                         Wilman Kala      WILMK      3161.35
88                      Wolski  Zajazd      WOLZA      3531.95

[89 rows x 3 columns]


**Performing Aggregations**  
I calculated summary statistics for different subsets of the data, focusing on determining
the total revenue generated by the company. By extracting and analyzing data from
relevant tables, I successfully obtained the total revenue figure, which serves as a critical
measure of the company's overall financial performance.

Calculate the total revenue generated by the company

In [ ]:
#3 Calculate the total revenue generated by the company
try:
    mycursor.execute("SELECT ROUND(SUM((UnitPrice*Quantity)-Discount),2) AS TotalCompanyRevenue FROM orderdetails")

    print("Total revenue generated by the company is:")

    for x in mycursor:
        print(x[0])

except mysql.connector.Error as error:
    print("Error executing the query:", error)

Total revenue generated by the company is:
1354458.59


**Performing Filtering**  
I filtered the data to concentrate on specific subsets, with a particular focus on finding
orders placed by a specific customer, identified by their contact name. This targeted
approach allowed for a detailed examination of that customer's transaction history,
providing valuable insights into their preferences, purchasing habits, and overall
engagement with the company.

In [ ]:
#3 Find orders placed by a specific customer identified by their contact name
try:
    mycursor.execute("SELECT * FROM orders NATURAL JOIN customers WHERE ContactName = 'Maria Anders'")

    title = [i[0] for i in mycursor.description]
    data=[]
    for x in mycursor:
        data.append(x)
    df = pd.DataFrame(data, columns=[title])
    print(df)

except mysql.connector.Error as error:
    print("Error executing the query:", error)

  CustomerID OrderID EmployeeID  OrderDate RequiredDate ShippedDate ShipVia  \
0      ALFKI   10643          6 1997-08-25   1997-09-22  1997-09-02       1   
1      ALFKI   10692          4 1997-10-03   1997-10-31  1997-10-13       2   
2      ALFKI   10702          4 1997-10-13   1997-11-24  1997-10-21       1   
3      ALFKI   10835          1 1998-01-15   1998-02-12  1998-01-21       3   
4      ALFKI   10952          1 1998-03-16   1998-04-27  1998-03-24       1   
5      ALFKI   11011          3 1998-04-09   1998-05-07  1998-04-13       1   

   Freight              ShipName    ShipAddress  ...          CompanyName  \
0  29.4600   Alfreds Futterkiste  Obere Str. 57  ...  Alfreds Futterkiste   
1  61.0200  Alfred-s Futterkiste  Obere Str. 57  ...  Alfreds Futterkiste   
2  23.9400  Alfred-s Futterkiste  Obere Str. 57  ...  Alfreds Futterkiste   
3  69.5300  Alfred-s Futterkiste  Obere Str. 57  ...  Alfreds Futterkiste   
4  40.4200  Alfred-s Futterkiste  Obere Str. 57  ...  Alfreds

**Joining Tables**  
I joined tables containing information about customers, orders, products, and employees
to gain insights into the relationships between different entities in the dataset. I identified
which products are currently on backorder by joining the Products and Order Details
tables. This query allowed me to pinpoint products that have a quantity ordered exceeding
the quantity in stock. These insights into backordered products can be crucial for inventory
management and ensuring timely customer fulfillment.

In [ ]:
#Identify which products are on backorder
#joins the Products and Order Details tables to identify which products are currently on backorder (i.e., have a quantity ordered that exceeds the quantity in stock).
#need to group all orders for same product and add total first then compare to units in stock
#only ones that haven't shipped yet, combine with order table
try:
    mycursor.execute('''SELECT products.ProductID, ProductName, SUM(Quantity) AS TotalOrdered, UnitsInStock
    FROM products
    JOIN orderdetails ON products.ProductID = orderdetails.ProductID
    JOIN orders ON orderdetails.OrderID =  orders.OrderID
    WHERE ShippedDate IS NULL
    GROUP BY products.ProductID
    HAVING TotalOrdered > UnitsInStock''')

    title = [i[0] for i in mycursor.description]
    data=[]
    for x in mycursor:
        data.append(x)
    df = pd.DataFrame(data, columns=[title])
    print(df)


except mysql.connector.Error as error:
    print("Error executing the query:", error)

   ProductID                      ProductName TotalOrdered UnitsInStock
0         28                 Rssle Sauerkraut           98           26
1         49                         Maxilaku           62           10
2         35                   Steeleye Stout           24           20
3         21              Sir Rodney's Scones           23            3
4         51            Manjimup Dried Apples           24           20
5         24                 Guaran Fantstica           30           20
6         60                Camembert Pierrot           73           19
7         13                            Konbu           44           24
8         17                     Alice Mutton           12            0
9         53                    Perth Pasties           10            0
10        43                      Ipoh Coffee           36           17
11         1                             Chai           40           39
12         2                            Chang           62      

**Using Subqueries**  
I used SQL subqueries to gain insights into specific subsets of the data, particularly focusing
on calculating the average order amount for customers who have placed more than five
orders. This analysis allowed me to identify the spending behavior of repeat customers and
understand their average order value. Such insights can be valuable for tailoring marketing
strategies and enhancing customer retention efforts.

In [ ]:
#Calculate the average order amount for customers who have placed more than one order:
#GROUP BY ORDER ID AND WHERE COUNT IS MORE THAN 5


try:
    mycursor.execute('''SELECT ShipName, CustomerID, ROUND(AVG((UnitPrice*Quantity)-Discount),2) AS AvgOrderValue
    FROM orders
    JOIN orderdetails ON orders.OrderID = orderdetails.OrderID
    WHERE CustomerID IN (SELECT CustomerID
        FROM orders
        GROUP BY CustomerID
        HAVING COUNT(CustomerID) > 5)
    GROUP BY CustomerID, ShipName''')

    title = [i[0] for i in mycursor.description]
    data=[]
    for x in mycursor:
        data.append(x)
    df = pd.DataFrame(data, columns=[title])
    print(df)


except mysql.connector.Error as error:
    print("Error executing the query:", error)

                   ShipName CustomerID AvgOrderValue
0         Toms Spezialitten      TOMSP        353.86
1             Hanari Carnes      HANAR       1065.66
2      Victuailles en stock      VICTE        397.48
3            Suprmes dlices      SUPRD        633.45
4         Chop-suey Chinese      CHOPS        585.74
..                      ...        ...           ...
59  Great Lakes Food Market      GREAL        895.96
60             Maison Dewey      MAISD        613.56
61              Wilman Kala      WILMK        185.96
62      Alfreds Futterkiste      ALFKI        362.00
63     Alfred-s Futterkiste      ALFKI        390.02

[64 rows x 3 columns]
